In [3]:
import sys
sys.path.append('../')
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from utility.file_utility import FileUtility




In [4]:
X=FileUtility.load_sparse_csr('../../datasets/bodysite/6-mers_rate_5000.npz')
Y=FileUtility.load_list('../../datasets/bodysite/data_config/labels_phen.txt')

In [24]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
onehot_y = np_utils.to_categorical(encoded_Y)

In [28]:
# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(512, input_dim=4096, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(100, input_dim=512, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [30]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score

skf = StratifiedKFold(n_splits=5, shuffle=True)

p_micro=[]
p_macro=[]
r_micro=[]
r_macro=[]
f1_micro=[]
f1_macro=[]

for train_index, valid_index in skf.split(X, Y):
    print ('New fold')
    X_train=X[train_index,:]
    y_train=onehot_y[train_index,:]
    y_class_train=encoded_Y[train_index]
    X_valid=X[valid_index,:]
    y_valid=onehot_y[valid_index,:]
    y_class_valid=encoded_Y[valid_index]
    model=baseline_model()
    history = model.fit(X_train, y_train, epochs=50, batch_size=100, validation_data=(X_valid, y_valid), verbose=0)
    pred=model.predict_classes(X_valid)
    f1_micro.append(f1_score(pred, y_class_valid, average='micro'))
    f1_macro.append(f1_score(pred, y_class_valid, average='macro'))
    p_micro.append(precision_score(pred, y_class_valid, average='micro'))
    p_macro.append(precision_score(pred, y_class_valid, average='macro'))
    r_micro.append(recall_score(pred, y_class_valid, average='micro'))
    r_macro.append(recall_score(pred, y_class_valid, average='macro'))

New fold


ValueError: setting an array element with a sequence.